# Najprostszy model

##Wstęp
Piosenki nie mają przypisanych gatunków, musimy temu zarządzić. Ponieważ wcześniej tych danych nie mieliśmy, nie możemy ocenić, jak dobrze dobrany jest gatunek piosenki.
Jakość podziału na grupy oceniamy na podstawie danych, które uzyskaliśmy: ile gatunków występuje w otrzymanym zbiore, jaka jest liczba piosenk poszcególnego gatunku. Liczba tych gatunków musi być zbliżona do liczby najczęściej występujących, "głównych" gatunków, również staramy się uzyskać bardziej zbalansowany podział na gatunki, staramy się unikać pojawiania się grup o pewnym gatunku, do którego należą 1-2 piosenki.


In [60]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from matplotlib import pyplot

In [61]:
!pip install pandas==1.5.2
!pip install joblib==1.2.0
from joblib import load, dump

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
tracks = load("/tracks_df_preprocessed(1)")

In [63]:
tracks.head(2)

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
0,48SFtLr5URCI97X2Ynfdnc,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0,291227,0,2yTUYhIf8fxptTIy3KLuJD,2014,0.603,0.517,6,-8.504,0.0235,0.695,0.000003,0.744,0.327,96.181,"[rock, classic uk pop, pop]"
1,1y0U0HAe5QfTRzOsz74bOt,My Foolish Heart,25,166080,0,338mC0yGyX0C9of8QMJ5hK,1950-01-01,0.313,0.116,0,-12.645,0.0319,0.953,0.331000,0.161,0.255,74.071,"[rock, classic uk pop, pop]"


In [64]:
# for clustering
tracks_only_features = tracks[['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']].copy()

In [65]:
# for determining the genre that characterizes the group
tracks_only_genres = tracks[['artists_genre']]

In [66]:
tracks_only_features.head(2)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.603,0.517,-8.504,0.0235,0.695,0.000003,0.744,0.327,96.181
1,0.313,0.116,-12.645,0.0319,0.953,0.331000,0.161,0.255,74.071


## Clustering: pierwsza próba
Zgodnie z założeniami, korzystamy z metody K średnich.
Na razie zapoznajemy się z tym, jak ogólnie będzie wyglądało przypisanie gatunków dla piosenek.
Optymalna liczba grup oraz parametry modelu grupowania zostaną dobrane później, na razie sprawdzane jest, czy da się uzyskać jakiekolwiek wyniki

In [67]:
number_of_clusters = 1000

In [68]:
X = np.array(tracks_only_features).astype(float)
model = KMeans(n_clusters=number_of_clusters)
clusters_for_tracks = model.fit_predict(X)

##Ustalenie gatunku,który odpowiada grupie
Na podstawie gatunków, w których artyści tworzą, za pomocą heurystyk, jaki gatunek występuje w grupie najczęściej, określamy gatunek całej grupy.

In [69]:
artists_genres = tracks_only_genres.values.tolist()

In [70]:
genres_per_cluster = []
for i in range(number_of_clusters):
  genres_per_cluster.append([])

Dla każdego utworu dostaliśmy listę gatunków, w których tworzy jej autor

In [71]:
print(artists_genres[0][0])

['rock', 'classic uk pop', 'pop']


In [72]:
for i in range(len(clusters_for_tracks)):
  genres = artists_genres[i][0]
  for genre in genres:
    # if (genre != 'pop'):
    #   genres_per_cluster[clusters_for_tracks[i]].append(genre)
    genres_per_cluster[clusters_for_tracks[i]].append(genre)
  

Dla każdej grupy mamy przypisane wszystkie wystąpienia gatunkow u autorów piosenek z tej grupy

In [73]:
print(genres_per_cluster[0])

['rock', 'pop', 'dance', 'easy_listening', 'jazz', 'latin', 'vintage tango', 'latin', 'pop', 'latin', 'jazz', 'musica popular mineira', 'samba', 'easy_listening', 'rock', 'acoustic', 'r&b', 'jazz', 'latin', 'brazilian rock', 'jazz', 'nova mpb', 'samba', 'easy_listening', 'jazz', 'blues', 'r&b', 'traditional_folk', 'nueva cancion', 'pop romantico', 'rock nacional', 'trova', 'classic j-pop', 'eurobeat', 'pop', 'australian rock', 'metal', 'rock', 'rock', 'pop', 'pop', 'traditional_folk', 'rock', 'pop', 'traditional_folk', 'taiwan pop', 'zhongguo feng', 'metal', 'rock', 'pop', 'pop', 'easy_listening', 'jazz', 'latin', 'boogaloo', 'jazz', 'latin', 'salsa international', 'rap', 'rock', 'acoustic', 'new_age', 'pop', 'corrido', 'norteno', 'latin', 'easy_listening', 'jazz', 'latin christian', 'latin worship', 'rap', 'reggaeton cristiano', 'acoustic', 'rock', 'madchester', 'classic bollywood', 'classic tollywood', 'desi pop', 'traditional_folk', 'sufi', 'classic bollywood', 'classic tollywood', 

In [74]:
def most_frequent(List):
  return max(set(List), key = List.count)

In [75]:
clusters_genres = []
for cluster in genres_per_cluster:
  clusters_genres.append(most_frequent(cluster))

Gatunek, występujący najczęściej w grupie określa gatunek grupy

In [76]:
print(len(set(clusters_genres)))
print(set(clusters_genres))

17
{'easy_listening', 'anime score', 'eurodance', 'metal', 'funk carioca', 'sertanejo pop', 'traditional_folk', 'classical', 'country', 'pop', 'rap', 'jazz', 'musica tradicional cubana', 'acoustic', 'latin', 'rock', 'new_age'}


## Wyniki
Nie najlepsze: bardzo niezbalansowane

In [77]:
tracks_genres=[]
for cluster in clusters_for_tracks:
  tracks_genres.append(clusters_genres[cluster])
tracks_with_genres = tracks.assign(genre=tracks_genres)

In [78]:
tracks_with_genres.head(5)

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre,genre
0,48SFtLr5URCI97X2Ynfdnc,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0,291227,0,2yTUYhIf8fxptTIy3KLuJD,2014,0.603,0.517,6,-8.504,0.0235,0.6950,0.000003,0.7440,0.327,96.181,"[rock, classic uk pop, pop]",pop
1,1y0U0HAe5QfTRzOsz74bOt,My Foolish Heart,25,166080,0,338mC0yGyX0C9of8QMJ5hK,1950-01-01,0.313,0.116,0,-12.645,0.0319,0.9530,0.331000,0.1610,0.255,74.071,"[rock, classic uk pop, pop]",pop
2,7ij5kN8jwXr8fZD54M0xb6,Aleni Aleni,51,235974,0,48CUA59SDed3IdCctKndud,2015,0.684,0.839,4,-6.457,0.0658,0.1200,0.000000,0.3540,0.580,128.051,"[rock, classic uk pop, pop]",pop
3,6tM2QQSXvoOAEfH0cARToU,A Contraluz,28,230560,0,2TkVZG5xW47ZC0227ftg3q,2009-10-28,0.439,0.709,2,-6.803,0.0339,0.0584,0.000000,0.0803,0.495,155.018,"[rock, classic uk pop, pop]",pop
4,6UIcN1tiiGdd7oMMzNvyaP,Time,53,297502,0,7lbSsjYACZHn1MSDXPxNF2,2020-05-08,0.479,0.478,5,-8.884,0.1100,0.0601,0.000007,0.0813,0.117,100.040,"[rock, classic uk pop, pop]",pop


In [79]:
tracks_with_genres.groupby('genre').count()

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
genre,,,,,,,,,,,,,,,,,,
acoustic,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33
anime score,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
classical,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
country,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
easy_listening,611,611,611,611,611,611,611,611,611,611,611,611,611,611,611,611,611,611
eurodance,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
funk carioca,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
jazz,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132
latin,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898,2898


##Zaawansowany clustering